In [1]:
%autosave 60
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
import sklearn.metrics
from sklearn.metrics import *
from sklearn.pipeline import make_pipeline

import pickle

Autosaving every 60 seconds
Automatically created module for IPython interactive environment


In [2]:
FEATURE_TYPE = 'deepmag' #['raw', 'deepmag']

if FEATURE_TYPE == 'raw':
    DATA_DIR = "C:/Wasif/PD Motor Feature Extraction/TASK2_FEATURES_04_21/"
    X_file = "x_repeat_removed_raw_pixels.npy"
    y_file = "y_repeat_removed_raw_pixels.npy"
    X_index_file = "index_repeat_removed.pickle"

elif FEATURE_TYPE == 'deepmag':
    DATA_DIR = "E:/Wasif/PDMotorFeatureExtraction/Task2_features_deepMAG/"
    X_file = "x_repeat_removed_deepmag.npy"
    y_file = "y_repeat_removed_deepmag.npy"
    X_index_file = "index_repeat_removed.pickle"

SEED = 1234
TRAIN_TEST_SPLIT = [0.80, 0.20]
TRAIN_TEST_DEV_SPLIT = [0.70, 0.15, 0.15]
NUM_FOLDS = 10
np.random.seed(seed=SEED)
IMG_HEIGHT = 492
IMG_WIDTH = 492

print(X_file)

x_repeat_removed_deepmag.npy


In [3]:
def load_dataset(DATA_DIR):
    X = np.load(DATA_DIR+X_file)
    #For now, only considering the frequency features
    X = X[:,(IMG_HEIGHT*IMG_WIDTH):]
    
    y = np.load(DATA_DIR+y_file)
    
    with open(DATA_DIR+X_index_file, 'rb') as handle:
        X_index = pickle.load(handle)  
    
    return (X, y, X_index)

In [4]:
def SMOTE_oversample(X,y):
    oversample = SMOTE()
    X, y = oversample.fit_resample(X, y)
    return (X,y)

In [5]:
def dataset_property(X, y):
    positives = np.sum(y==1.0)
    negatives = np.sum(y==0.0)
    print(positives, negatives)
    return

In [6]:
def train(X_train, y_train, model):
    (X,y) = SMOTE_oversample(X_train,y_train)
    #(X, y) = (X_train, y_train)
    clf = make_pipeline(StandardScaler(), model)
    clf.fit(X, y)
    return clf

In [7]:
def evaluate(y, pred_y):
    performance = {}
    performance['keys'] = ['accuracy', 'auc-roc', 'f1', 'precision', 'recall', 'balanced_accuracy']
    performance['accuracy'] = accuracy_score(y, pred_y)
    performance['auc-roc'] = roc_auc_score(y, pred_y)
    performance['f1'] = f1_score(y, pred_y)
    performance['precision'] = precision_score(y, pred_y)
    performance['recall'] = recall_score(y, pred_y)
    performance['balanced_accuracy'] = balanced_accuracy_score(y, pred_y)
    
    return performance

In [8]:
def k_fold_CV(model, X, y, X_index, k):
    #k iterations
    #Split into k fold, (k-1) = train, last = test
    #Evaluate on test
    #Report average
    performance = {}
    performance['keys'] = ['accuracy', 'auc-roc', 'f1', 'precision', 'recall', 'balanced_accuracy']
    performance['accuracy'] = 0.0
    performance['auc-roc'] = 0.0
    performance['f1'] = 0.0
    performance['precision'] = 0.0
    performance['recall'] = 0.0
    performance['balanced_accuracy'] = 0.0
    
    N = X.shape[0]
    samples_per_fold = (int)(N/k)
    
    X_folds = []
    y_folds = []
    X_index_folds = []
    
    shuffled_indices =np.arange(0,N) 
    np.random.shuffle(shuffled_indices)
    
    for fold_no in range(0,k):
        #fold_no set is the test set
        if fold_no==(k-1):
            test_indices = shuffled_indices[(samples_per_fold*fold_no): N]
        else:
            test_indices = shuffled_indices[(samples_per_fold*fold_no): (samples_per_fold*(fold_no+1))]
        
        train_indices = [i for i in range(0,N) if i not in test_indices]
        
        (X_train, y_train) = (X[train_indices,:], y[train_indices])
        (X_test, y_test) = (X[test_indices,:], y[test_indices])

        X_index_train = []
        X_index_test = []

        for i in train_indices:
            X_index_train.append(X_index[i])

        for i in test_indices:
            X_index_test.append(X_index[i])
        
        clf = train(X_train, y_train, model)
        
        #test performance using X_test, y_test
        fold_performance = evaluate(y_test, clf.predict(X_test))
        print("Fold %d"%(fold_no))
        print(fold_performance)
        
        for key in fold_performance['keys']:
            performance[key] +=fold_performance[key]
        
        #Show the samples with wrong predictions?
    
    for key in fold_performance['keys']:
            performance[key]  = performance[key]/k
    
    return performance

In [9]:
def train_test_split():
    (X, y, X_index) = load_dataset(DATA_DIR)
    N = X.shape[0]
    N_train = (int)(N*TRAIN_TEST_SPLIT[0])
    N_test = N - N_train
    
    #print(N, N_train, N_test)
    
    test_indices = np.random.choice(N, N_test, replace=False)
    train_indices = [i for i in range(0,N) if i not in test_indices]
    
    (X_train, y_train) = (X[train_indices,:], y[train_indices])
    (X_test, y_test) = (X[test_indices,:], y[test_indices])
    
    X_index_train = []
    X_index_test = []
    
    for i in train_indices:
        X_index_train.append(X_index[i])
        
    for i in test_indices:
        X_index_test.append(X_index[i])
    
    return (X_train, y_train, X_index_train, X_test, y_test, X_index_test)

In [10]:
#train_test_split()

In [11]:
def train_test_dev_split():
    (X, y, X_index) = load_dataset(DATA_DIR)
    N = X.shape[0]
    N_train = (int)(N*TRAIN_TEST_DEV_SPLIT[0])
    N_test = (int)(N*TRAIN_TEST_DEV_SPLIT[1])
    N_dev = N - N_train - N_test
    
    print(N_train, N_test, N_dev)
    
    test_dev_indices = np.random.choice(N, (N_test+N_dev), replace=False)
    
    test_indices = test_dev_indices[0:N_test]
    dev_indices = test_dev_indices[N_test:]
    train_indices = [i for i in range(0,N) if i not in test_dev_indices]
    
    (X_train, y_train) = (X[train_indices,:], y[train_indices])
    (X_test, y_test) = (X[test_indices,:], y[test_indices])
    (X_dev, y_dev) = (X[dev_indices,:], y[dev_indices])
    
    X_index_train = []
    X_index_test = []
    X_index_dev = []
    
    for i in train_indices:
        X_index_train.append(X_index[i])
        
    for i in test_indices:
        X_index_test.append(X_index[i])
        
    for i in dev_indices:
        X_index_dev.append(X_index[i])
    
    return (X_train, y_train, X_index_train, X_test, y_test, X_index_test, X_dev, y_dev, X_index_dev)

In [12]:
#train_test_dev_split()

In [13]:
(X, y, X_index) = load_dataset(DATA_DIR)
assert(X.shape[0]==y.shape[0] and y.shape[0]==len(X_index))
print("Dataset Loaded")

Dataset Loaded


In [14]:
#With raw features and full dataset: 0.6556543837357052

#train model using X_train, y_train
model = sklearn.svm.SVC(C=100)
model.probability = True
#clf = train(X, y, model)
model.fit(X, y)

# save the model to disk
filename = 'E:/Wasif/PDMotorFeatureExtraction/deepmag_svm_fullfit_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [20]:
loaded_model = pickle.load(open(filename, 'rb'))
prediction = loaded_model.predict(X)
prediction_prob = loaded_model.predict_proba(X)
print(prediction_prob[0].shape)
print(prediction.shape)
print(y.shape)


(2,)
(762,)
(762,)


In [26]:
from sklearn.metrics import accuracy_score, roc_auc_score
accuracy = accuracy_score(y,prediction)
roc = roc_auc_score(y, prediction_prob[:,0].flatten())
print(accuracy, roc)

0.6771653543307087 0.7516127593792734
